Thiết lập

In [53]:
import os
from pathlib import Path
import numpy as np
import pandas as pd, re, os


Đọc dữ liệu

In [54]:
micro = pd.read_csv('../raw/Microblogs.csv', encoding='latin-1')
weather = pd.read_csv('../raw/Weather.csv', encoding='latin-1')

print('Microblogs:', micro.shape)
print('Weather:', weather.shape)
micro.head()


Microblogs: (1023077, 4)
Weather: (21, 4)


,ID,Created_at,Location,text
0,3,5/18/2011 13:26,42.22717 93.33772,this convention filled with technology could b...
1,3,5/10/2011 9:22,42.18881 93.35642,4 orang top scorer skrg 3 di antara nya pake ...
2,3,5/14/2011 9:22,42.22479 93.35922,Nike 'gana' el Mundial al patrocinador oficial...
3,3,5/6/2011 9:22,42.2469 93.32527,Positiiff mau nabung beli kaos adidas aslii...
4,3,5/19/2011 9:22,42.2354 93.35642,I kick it like Adidas


Làm sạch dữ liệu, nội dung văn bản  

In [55]:
micro['text_clean'] = micro['text'].fillna('').astype(str)
micro['text_clean'] = micro['text_clean'].str.lower()
micro['text_clean'] = micro['text_clean'].apply(lambda x: re.sub(r'\s+', ' ', x.strip()))
micro[['text', 'text_clean']].head()




,text,text_clean
0,this convention filled with technology could b...,this convention filled with technology could b...
1,4 orang top scorer skrg 3 di antara nya pake ...,4 orang top scorer skrg 3 di antara nya pake a...
2,Nike 'gana' el Mundial al patrocinador oficial...,nike 'gana' el mundial al patrocinador oficial...
3,Positiiff mau nabung beli kaos adidas aslii...,positiiff mau nabung beli kaos adidas aslii mu...
4,I kick it like Adidas,i kick it like adidas


Chuẩn hóa thời gian         
Các xử lí tiếp theo sẽ sử dụng  **micro_sub** và **weather_sub**

In [56]:
# Giới hạn dữ liệu của nhóm 2.4
start = "2011-05-12 00:00:00"
end   = "2011-05-15 23:59:59"

micro = micro.rename(columns={'Created_at': 'timestamp'})
micro['timestamp'] = pd.to_datetime(micro['timestamp'], errors='coerce')
micro_sub = micro[(micro['timestamp'] >= start) & (micro['timestamp'] <= end)].copy()
print('Khoảng thời gian dữ liệu:', micro['timestamp'].min(), '-', micro['timestamp'].max())

weather = weather.rename(columns={'Date': 'timestamp'})
weather['timestamp'] = pd.to_datetime(weather['timestamp'], errors='coerce')
weather_sub = weather[(weather['timestamp'] >= start) & (weather['timestamp'] <= end)].copy()
print('Weather:', weather_sub['timestamp'].min(), '-', weather_sub['timestamp'].max())


Khoảng thời gian dữ liệu: 2011-04-30 00:00:00 - 2011-05-20 23:59:00
Weather: 2011-05-12 00:00:00 - 2011-05-15 00:00:00


Chuẩn hóa thời tiết

In [57]:
weather_sub['Weather'] = weather_sub['Weather'].str.strip().str.upper()
weather_sub['Wind_Direction'] = weather_sub['Wind_Direction'].str.strip().str.upper()

weather_map = {
    'CLEAR': 'SUNNY', 'SUN': 'SUNNY', 'RAINY': 'RAIN', 'SHOWERS': 'RAIN',
    'OVERCAST': 'CLOUDY', 'CLOUDY': 'CLOUDY', 'STORMY': 'STORM', 'FOGGY': 'FOG'
}
weather_sub['Weather'] = weather_sub['Weather'].replace(weather_map)

# Loại giá trị không nằm trong danh mục
valid_weather = ['SUNNY','CLOUDY','RAIN','STORM','FOG']
weather_sub.loc[~weather_sub['Weather'].isin(valid_weather), 'Weather'] = 'UNKNOWN'


Chuẩn hóa vị trí địa lí

In [58]:
import re
def parse_location(s):
    try:
        lat, lon = re.findall(r"[-+]?\d*\.\d+|\d+", str(s))
        lat, lon = float(lat), float(lon)
        if 'S' in s: lat = -lat
        if 'W' in s: lon = -lon
        return lat, lon
    except:
        return (None, None)

micro_sub[['lat','lon']] = micro_sub['Location'].apply(lambda s: pd.Series(parse_location(s)))


Gắn cờ lỗi dữ liệu

In [59]:
# Tạo cột flag mặc định
micro_sub['flag'] = 'ok'

# 1. Thiếu nội dung text
micro_sub.loc[micro_sub['text_clean'].str.strip() == '', 'flag'] = 'missing_text'

# 2. Thiếu vị trí
if 'Location' in micro_sub.columns:
    micro_sub.loc[micro_sub['Location'].isna(), 'flag'] = 'missing_location'

# 3. Thiếu hoặc lỗi thời gian
micro_sub.loc[micro_sub['timestamp'].isna(), 'flag'] = 'bad_timestamp'

#################################

weather_sub['flag'] = 'ok'

# 1. Thiếu hoặc lỗi thời gian
weather_sub.loc[weather_sub['timestamp'].isna(), 'flag'] = 'bad_timestamp'

# 2. Thiếu kiểu thời tiết
if 'weather' in weather_sub.columns:
    weather_sub.loc[weather_sub['weather'].isna() | (weather_sub['weather'].astype(str).str.strip() == ''), 'flag'] = 'missing_weather'

# 3. Thiếu hướng gió
if 'wind_direction' in weather_sub.columns:
    weather_sub.loc[weather_sub['wind_direction'].isna() | (weather_sub['wind_direction'].astype(str).str.strip() == ''), 'flag'] = 'missing_wind_direction'

# 4. Tốc độ gió âm hoặc không hợp lý
if 'average_wind_speed' in weather_sub.columns:
    weather_sub.loc[weather_sub['average_wind_speed'] < 0, 'flag'] = 'negative_wind_speed'
    weather_sub.loc[weather_sub['average_wind_speed'] > 200, 'flag'] = 'abnormal_wind_speed'  # tuỳ chỉnh ngưỡng


Báo cáo tác động chuẩn hóa

In [60]:
# QA microblogs
qa_micro = micro_sub['flag'].value_counts().reset_index()
qa_micro.columns = ['flag', 'count']
qa_micro['percent'] = (qa_micro['count'] / len(micro_sub) * 100).round(2)
qa_micro['table'] = 'microblogs'

# QA weather
qa_weather = weather_sub['flag'].value_counts().reset_index()
qa_weather.columns = ['flag', 'count']
qa_weather['percent'] = (qa_weather['count'] / len(weather_sub) * 100).round(2)
qa_weather['table'] = 'weather'

# Gộp chung
qa_summary = pd.concat([qa_micro, qa_weather], ignore_index=True)
qa_summary = qa_summary[['table', 'flag', 'count', 'percent']]

# Ghi ra một file duy nhất
qa_summary.to_csv('../reports/qa_summary.csv', index=False, encoding='utf-8')

print("Đã lưu tổng hợp QA vào ../reports/qa_summary.csv")
qa_summary


Đã lưu tổng hợp QA vào ../reports/qa_summary.csv


,table,flag,count,percent
0,microblogs,ok,182386,99.98
1,microblogs,missing_text,36,0.02
2,weather,ok,4,100.00


Lưu dữ liệu sau khi làm sạch **microblogs_clean.csv** và **weather_clean.csv**

In [61]:
# Đảm bảo có thư mục processed/
import os
os.makedirs('../processed', exist_ok=True)

# Lưu microblogs và weather sau khi làm sạch
micro_sub.to_csv('../processed/microblogs_clean.csv', index=False, encoding='utf-8')
weather_sub.to_csv('../processed/weather_clean.csv', index=False, encoding='utf-8')

print("Đã lưu vào thư mục")


Đã lưu vào thư mục
